In [1]:
!pip install uv -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 34.3 MB/s eta 0:00:00


In [2]:
!pip install google-adk google-generativeai pyngrok -q

In [3]:
from google.colab import userdata
import os
from termcolor import colored

In [4]:
from google.colab import userdata
import os
from termcolor import colored

# Get API key and MCP server URL from Colab secrets
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
    print(colored("Google API Key loaded", "green"))
except Exception as e:
    print(colored("GOOGLE_API_KEY not found in secrets", "red"))
    print(colored("   Please add your Google API key to Colab secrets", "yellow"))
    print(colored("   Get one at: https://aistudio.google.com/app/apikey", "yellow"))
    raise

try:
    MCP_SERVER_URL = userdata.get('MCP_SERVER_URL')
    print(colored(f"MCP Server URL loaded: {MCP_SERVER_URL}", "green"))
except Exception as e:
    print(colored("MCP_SERVER_URL not found in secrets", "red"))
    print(colored("   Please add your MCP server URL to Colab secrets", "yellow"))
    print(colored("   Format: https://xxxx.ngrok.io/mcp", "yellow"))
    print(colored("   Run the mcp_customer_demo.ipynb notebook first to get this URL", "yellow"))
    raise

print()
print(colored("Configuration complete!", "cyan", attrs=["bold"]))

Google API Key loaded
MCP Server URL loaded: https://polar-nonsolubly-madden.ngrok-free.dev/mcp

Configuration complete!


In [5]:
from google.adk.agents import LlmAgent
from google.adk.tools.mcp_tool import McpToolset, StreamableHTTPConnectionParams

print(colored("Creating ADK Agent with MCP tools...", "cyan"))

# Create the agent
customer_agent = LlmAgent(
    model="gemini-2.5-flash",
    name="customer_management_agent",
    tools=[
        McpToolset(
            connection_params=StreamableHTTPConnectionParams(
                url=MCP_SERVER_URL
            )
        )
    ],
    instruction="""You are a helpful customer management assistant.

You have access to a customer database through MCP tools. Use these tools to help users:
- Get information about specific customers
- List all customers or filter by status
- Add new customers
- Update customer information
- Disable or activate customer accounts

Always provide clear, friendly responses. When performing operations, explain what you're doing.
Format customer information in a clear, readable way."""
)

print(colored("Agent created successfully!", "green", attrs=["bold"]))
print()
print(colored("Agent Details:", "cyan"))
print(f"   Name: {customer_agent.name}")
print(f"   Model: gemini-2.5-flash")
print(f"   Tools: MCPToolSet connected to {MCP_SERVER_URL}")
print()
print(colored("The agent can now use all customer management tools!", "yellow"))

/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


Creating ADK Agent with MCP tools...
Agent created successfully!

Agent Details:
   Name: customer_management_agent
   Model: gemini-2.5-flash
   Tools: MCPToolSet connected to https://polar-nonsolubly-madden.ngrok-free.dev/mcp

The agent can now use all customer management tools!


In [6]:
from google.adk import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
import warnings
import logging
import sys

# Suppress all warnings and errors for cleaner output
warnings.filterwarnings('ignore')
logging.getLogger('google_adk').setLevel(logging.CRITICAL)
logging.getLogger('google_genai').setLevel(logging.CRITICAL)
logging.getLogger('asyncio').setLevel(logging.CRITICAL)

In [7]:
async def ask_agent(query: str, show_usage: bool = False):
    """
    Send a query to the agent and display the response using Google ADK Runner.

    Args:
        query: The question or request to send to the agent
        show_usage: If True, show token usage statistics
    """
    print(colored("="*70, "magenta"))
    print(colored(f"USER: {query}", "cyan", attrs=["bold"]))
    print(colored("="*70, "magenta"))
    print()

    SESSION_ID = "session_456"
    USER_ID = "user_123"
    app_name = "customer_management_agent"

    try:
        # Create content from user query
        content = types.Content(
            role="user",
            parts=[types.Part(text=query)]
        )

        # Create a session service
        session_service = InMemorySessionService()

        # Create session asynchronously
        session = await session_service.create_session(
            app_name=app_name,
            user_id=USER_ID,
            session_id=SESSION_ID
        )

        # Create a runner
        runner = Runner(
            agent=customer_agent,
            session_service=session_service,
            app_name=app_name
        )

        # Run the agent with the runner
        events = runner.run(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        )

        # Process events to get the final response
        agent_response = None
        full_response = None
        for event in events:
            if event.is_final_response():
                agent_response = event.content.parts[0].text
                full_response = event
                break

        if agent_response is None:
            agent_response = "No response received."

        print(colored("AGENT:", "green", attrs=["bold"]))
        print(agent_response)
        print()

        return agent_response

    except Exception as e:
        print(colored(f"Error: {e}", "red"))
        import traceback
        traceback.print_exc()
        print()
        return None

print(colored("Helper function defined (Google ADK Runner version)", "green"))

Helper function defined (Google ADK Runner version)


In [8]:
await ask_agent("Can you show me all the customers?")

USER: Can you show me all the customers?

AGENT:
Here is a list of all your customers:

*   **ID: 4**
    *   Name: Alice Williams
    *   Email: alice.w@techcorp.com
    *   Phone: +1-555-0104
    *   Status: active
    *   Created At: 2025-11-17 21:35:02
    *   Updated At: 2025-11-17 21:35:02
*   **ID: 3**
    *   Name: Bob Johnson
    *   Email: bob.johnson@example.com
    *   Phone: +1-555-0103
    *   Status: disabled
    *   Created At: 2025-11-17 21:35:02
    *   Updated At: 2025-11-17 21:35:02
*   **ID: 5**
    *   Name: Charlie Brown
    *   Email: charlie.brown@email.com
    *   Phone: +1-555-0105
    *   Status: active
    *   Created At: 2025-11-17 21:35:02
    *   Updated At: 2025-11-17 21:35:02
*   **ID: 6**
    *   Name: Diana Prince
    *   Email: diana.prince@company.org
    *   Phone: +1-555-0106
    *   Status: active
    *   Created At: 2025-11-17 21:35:02
    *   Updated At: 2025-11-17 21:35:02
*   **ID: 7**
    *   Name: Edward Norton
    *   Email: e.norton@busi

'Here is a list of all your customers:\n\n*   **ID: 4**\n    *   Name: Alice Williams\n    *   Email: alice.w@techcorp.com\n    *   Phone: +1-555-0104\n    *   Status: active\n    *   Created At: 2025-11-17 21:35:02\n    *   Updated At: 2025-11-17 21:35:02\n*   **ID: 3**\n    *   Name: Bob Johnson\n    *   Email: bob.johnson@example.com\n    *   Phone: +1-555-0103\n    *   Status: disabled\n    *   Created At: 2025-11-17 21:35:02\n    *   Updated At: 2025-11-17 21:35:02\n*   **ID: 5**\n    *   Name: Charlie Brown\n    *   Email: charlie.brown@email.com\n    *   Phone: +1-555-0105\n    *   Status: active\n    *   Created At: 2025-11-17 21:35:02\n    *   Updated At: 2025-11-17 21:35:02\n*   **ID: 6**\n    *   Name: Diana Prince\n    *   Email: diana.prince@company.org\n    *   Phone: +1-555-0106\n    *   Status: active\n    *   Created At: 2025-11-17 21:35:02\n    *   Updated At: 2025-11-17 21:35:02\n*   **ID: 7**\n    *   Name: Edward Norton\n    *   Email: e.norton@business.net\n    * 